In [0]:
import pandas as pd
import pickle
import numpy as np
import librosa
#!pip install librosa

In [0]:
prosdic_features = pd.read_csv('new_data_with_prosodic.csv')

In [0]:
prosdic_features.head()

,Unnamed: 0,Unnamed: 0.1,emotions,gender,emotional_intensity,repetions,voice_channel,log_energy,zero_rate_cross,pitch,spec_center,spec_contrast,chroma
0,0,0,1,1,1,1,1,1.210089,0.276547,425.633360,2561.615654,18.579795,0.588629
1,1,1,1,1,1,2,1,0.953821,0.232797,135.463690,2389.013861,18.979816,0.561213
2,2,2,1,1,1,1,1,2.360058,0.276442,263.613601,2364.081624,18.604097,0.568334
3,3,3,1,1,1,2,1,1.963170,0.241621,400.331243,2265.299648,19.000360,0.572616
4,4,4,2,1,1,1,1,0.282539,0.232923,151.828945,2361.018676,18.629941,0.535452


In [0]:
x = prosdic_features.pitch.values #returns a numpy array
temp = (x-np.mean(x))/np.std(x)
prosdic_features['pitch'] = temp

x = prosdic_features.log_energy.values #returns a numpy array
temp = (x-np.mean(x))/np.std(x)
prosdic_features['log_energy'] = temp

In [0]:
prosdic_features = prosdic_features.drop(prosdic_features.columns[0:2],axis = 1)
prosdic_features.head()

,emotions,gender,emotional_intensity,repetions,voice_channel,log_energy,zero_rate_cross,pitch,spec_center,spec_contrast,chroma
0,1,1,1,1,1,0.609594,0.276547,0.129385,2561.615654,18.579795,0.588629
1,1,1,1,2,1,0.329755,0.232797,-1.063384,2389.013861,18.979816,0.561213
2,1,1,1,1,1,1.865338,0.276442,-0.536612,2364.081624,18.604097,0.568334
3,1,1,1,2,1,1.431944,0.241621,0.025379,2265.299648,19.000360,0.572616
4,2,1,1,1,1,-0.403272,0.232923,-0.996113,2361.018676,18.629941,0.535452


In [0]:
with open("mfcc_data", 'rb') as f:
    mfcc1 = pickle.load(f)
with open("pitch_data", 'rb') as f:
    pitchnew = pickle.load( f)
"""

with open("spectral_center_data", 'rb') as f:
    spectral_center1 = pickle.load(f)
with open("chroma_data", 'rb') as f:
    chroma1 = pickle.load(f)
with open("spectral_contrast_data", 'rb') as f:
    spectral_contrast1 = pickle.load(f)

with open("pitch1_data", 'rb') as f:
    pitch1new = pickle.load( f)
    
"""

'\n\nwith open("spectral_center_data", \'rb\') as f:\n    spectral_center1 = pickle.load(f)\nwith open("chroma_data", \'rb\') as f:\n    chroma1 = pickle.load(f)\nwith open("spectral_contrast_data", \'rb\') as f:\n    spectral_contrast1 = pickle.load(f)\n\nwith open("pitch1_data", \'rb\') as f:\n    pitch1new = pickle.load( f)\n    \n'

In [0]:
mfcc = []
spectral_center = []
chroma = []
spectral_contrast = []
pitch = []

for i in range(len(mfcc1)):
  mfcc.append(np.array(mfcc1[i]))
  pitch.append(np.array(pitchnew[i]))

In [0]:
from sklearn.preprocessing import StandardScaler 
for i in range(len(mfcc)):
  mfcc[i]= StandardScaler().fit_transform(mfcc[i])
 # chroma[i]= StandardScaler().fit_transform(chroma[i])
 # spectral_contrast[i]= StandardScaler().fit_transform(spectral_contrast[i])

In [0]:
maxmfcc = []
maxpitch = []
for i in range(len(mfcc1)):
  maxmfcc.append((mfcc[i]).shape[1])
  #maxpitch.append(len(pitch[i]))

In [0]:
mfcc_max = np.max(maxmfcc)

mfcc_max

165

In [0]:
def pad(mfcc):
  mfcc_new = []
  for i in range(len(mfcc)):
    arr = []
    for j in range(len(mfcc[i])):
      val = np.pad(mfcc[i][j], (0, 200-len(mfcc[i][j])), 'constant')
      arr.append(val)
    mfcc_new.append(np.array(arr))
  return mfcc_new

In [0]:
mfcc = pad(mfcc)


In [0]:
mfcc[0].shape

(20, 200)

In [0]:
mfcc = np.array(mfcc).reshape(len(mfcc),20,200)

In [0]:
mfcc.shape

(1440, 20, 200)

In [0]:
def append_features(mfcc_data,chroma_data):
  final = []
  for i in range(mfcc_data.shape[0]):
    new = np.concatenate((mfcc_data[i,:,:],chroma_data[i,:,:]))
    final.append(new)
  return np.array(final)

In [0]:
def append_values(data,feature):
  new_data = []
  for i in range(data.shape[0]):
    value = prosdic_features.iloc[i][feature]
    line = np.full((1,data[i].shape[1]),value)[0]
    line = line.reshape(1,data[i].shape[1])
    data1 = np.concatenate((data[i],line))
    new_data.append(data1)
  return np.array(new_data)

In [0]:
mfcc_with_pitch = append_values(mfcc,'pitch')

In [0]:
mfcc_with_pitch.shape

(1440, 21, 200)

In [0]:
mfcc_with_pitch_logenergy = append_values(mfcc_with_pitch,'log_energy')

In [0]:
mfcc_with_pitch_logenergy.shape

(1440, 22, 200)

In [0]:

#t1 = append_features(mfcc,chroma)
#t2 = append_features(t1,spectral_contrast)

In [0]:
mfcc.shape

(1440, 20, 200)

In [0]:
specs = np.asarray(mfcc_with_pitch_logenergy).reshape(len(mfcc_with_pitch_logenergy),22,200,1)
#specs = np.asarray(t2).reshape(len(t2),39,165,1)
features = np.concatenate((specs, np.zeros(np.shape(specs))), axis = 3)
for i in range(len(features)):
  features[i, :, :, 1] = librosa.feature.delta(features[i, :, :, 0])

In [0]:
features_speech = features

In [0]:
y_data = pd.read_csv('processed_new_data.csv')
y_data.drop([y_data.columns[0]],inplace = True,axis = 1)
y_data.head()

,emotional_intensity,repetions,neutral,calm,happy,sad,angry,fearful,disgust,surprised,male,female
0,1,1,1,0,0,0,0,0,0,0,1,0
1,1,2,1,0,0,0,0,0,0,0,1,0
2,1,1,1,0,0,0,0,0,0,0,1,0
3,1,2,1,0,0,0,0,0,0,0,1,0
4,1,1,0,1,0,0,0,0,0,0,1,0


In [0]:
y = y_data.loc[:,'neutral':'surprised']
y.head()

,neutral,calm,happy,sad,angry,fearful,disgust,surprised
0,1,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0


In [0]:
y_speech = y

In [0]:
y_speech.shape

(1440, 8)

In [0]:
with open("mfcc_song_data", 'rb') as f:
    mfcc_song = pickle.load(f)
with open("pitch_song_data", 'rb') as f:
    pitch_song = pickle.load( f)

In [0]:
mfcc_songs = []
pitch_songs = []
for i in range(len(mfcc_song)):
  mfcc_songs.append(np.array(mfcc_song[i]))
  pitch_songs.append(np.array(pitch_song[i]))

In [0]:
pitch_songs= np.array(pitch_songs).ravel()


In [0]:
pitch_songs.shape

(1012,)

In [0]:
x = pitch_songs #returns a numpy array
temp = (x-np.mean(x))/np.std(x)


In [0]:
pitch_songs = temp


In [0]:
from sklearn.preprocessing import StandardScaler 
for i in range(len(mfcc_songs)):
  mfcc_songs[i]= StandardScaler().fit_transform(mfcc_songs[i])
  

In [0]:
maxmfcc_songs = []
maxpitch_songs = []
for i in range(len(mfcc_songs)):
  maxmfcc.append((mfcc_songs[i]).shape[1])

In [0]:
mfcc_max = np.max(maxmfcc)
mfcc_max

200

In [0]:
def pad(mfcc):
  mfcc_new = []
  for i in range(len(mfcc)):
    arr = []
    for j in range(len(mfcc[i])):
      val = np.pad(mfcc[i][j], (0, 200-len(mfcc[i][j])), 'constant')
      arr.append(val)
    mfcc_new.append(np.array(arr))
  return mfcc_new

In [0]:
mfcc_songs = pad(mfcc_songs)

In [0]:
mfcc_songs = np.array(mfcc_songs).reshape(len(mfcc_songs),20,200)

In [0]:
mfcc_songs.shape

(1012, 20, 200)

In [0]:
def append_values_songs(data,feature):
  new_data = []
  for i in range(data.shape[0]):
    value = pitch_songs[i]
    print (value)
    line = np.full((1,data[i].shape[1]),value)[0]
    line = line.reshape(1,data[i].shape[1])
    data1 = np.concatenate((data[i],line))
    new_data.append(data1)
  return np.array(new_data)

In [0]:
combined_songs = append_values_songs(mfcc_songs,pitch_songs)
combined_songs.shape

0.38708711532312695
-1.3367874430089115
-1.3327934012289069
-1.3387966924568644
-1.342817137100298
-1.3381966408403068
-1.360063590802275
-1.1108766785823565
-1.1170823996088288
-1.1095455313599127
-1.1092882530871935
-1.1047740057879085
0.37078200508901976
-0.48030709083654716
0.35126237264804516
1.1237052036690058
0.42253139105814674
0.3683304240457432
0.3903127507449009
0.4606238196347524
-1.3477724546995025
-1.3504282808237222
-1.3459239304692951
0.27794578027187766
-1.3556342343545116
-1.3574748223633242
-0.15345209071443083
-1.1558963305872691
-1.1574673727726092
0.3285316524128034
0.29889525737891337
-1.3281297595160166
1.2295896907909294
-1.13538354651286
1.1898007952002594
0.14742831796163353
0.28884298666683433
-1.3575145860969131
-1.1640489760422041
-1.3499458104436965
-1.145002488373729
0.3760282507695899
0.4397005151936566
0.44724867972513416
1.195014123342858
2.855958144109288
2.854775100952704
1.1643679153943067
-0.5163223386189839
-0.5288151657403303
-0.5246124706290349

(1012, 21, 200)

In [0]:
combined_songs.shape

(1012, 21, 200)

In [0]:
prosdic_features_songs = pd.read_csv('song_new_data_with_prosodic.csv')

In [0]:
prosdic_features_songs.head()

,Unnamed: 0,Unnamed: 0.1,emotional_intensity,repetions,neutral,calm,happy,sad,angry,fearful,male,female,log_energy,zero_rate_cross,pitch
0,0,0,1,1,1,0,0,0,0,0,1,0,1.435604,0.275862,349.147735
1,1,1,1,2,1,0,0,0,0,0,1,0,0.984460,0.251336,356.027898
2,2,2,1,1,1,0,0,0,0,0,1,0,1.236308,0.235192,350.963671
3,3,3,1,2,1,0,0,0,0,0,1,0,1.133245,0.278376,356.090107
4,4,4,1,1,0,1,0,0,0,0,1,0,0.671422,0.299620,345.419743


In [0]:
prosdic_features_songs = prosdic_features_songs.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1)
prosdic_features_songs.head()

,emotional_intensity,repetions,neutral,calm,happy,sad,angry,fearful,male,female,log_energy,zero_rate_cross,pitch
0,1,1,1,0,0,0,0,0,1,0,1.435604,0.275862,349.147735
1,1,2,1,0,0,0,0,0,1,0,0.984460,0.251336,356.027898
2,1,1,1,0,0,0,0,0,1,0,1.236308,0.235192,350.963671
3,1,2,1,0,0,0,0,0,1,0,1.133245,0.278376,356.090107
4,1,1,0,1,0,0,0,0,1,0,0.671422,0.299620,345.419743


In [0]:
y_speech.columns

Index(['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful', 'disgust',
       'surprised'],
      dtype='object')

In [0]:
y_songs = prosdic_features_songs.drop(['emotional_intensity','repetions','male','female','log_energy','zero_rate_cross','pitch'],axis=1)

y_songs['disgust'] = 0
y_songs['surprised'] = 0
y_songs.columns

Index(['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful', 'disgust',
       'surprised'],
      dtype='object')

In [0]:
def append_values_songs_energy(data,feature):
  new_data = []
  for i in range(data.shape[0]):
    value = prosdic_features_songs.iloc[i][feature]
    line = np.full((1,data[i].shape[1]),value)[0]
    line = line.reshape(1,data[i].shape[1])
    data1 = np.concatenate((data[i],line))
    new_data.append(data1)
  return np.array(new_data)

In [0]:
combined_songs = append_values_songs_energy(combined_songs,'log_energy')
combined_songs.shape

(1012, 22, 200)

In [0]:
features_songs = combined_songs
features_songs.shape

(1012, 22, 200)

In [0]:

specs = np.asarray(features_songs).reshape(len(features_songs),22,200,1)
#specs = np.asarray(features_songs).reshape(len(features_songs),21,200,1)
features = np.concatenate((specs, np.zeros(np.shape(specs))), axis = 3)
for i in range(len(features)):
  features[i, :, :, 1] = librosa.feature.delta(features[i, :, :, 0])

In [0]:
features_songs = features
features_songs.shape

(1012, 22, 200, 2)

In [0]:
features_speech.shape

(1440, 22, 200, 2)

In [0]:
features_combined = []
features_combined = np.append(features_speech,features_songs,axis=0)
features_combined.shape

(2452, 22, 200, 2)

In [0]:
y_speech.shape

(1440, 8)

In [0]:
y_combined = np.append(y_speech,y_songs,axis=0)
y_combined.shape

(2452, 8)

In [0]:
features = features_combined
y = y_combined

In [0]:
features.shape
y.shape

(2452, 8)

In [0]:
features = features.reshape(features.shape[0],features.shape[1]*features.shape[2]*features.shape[3])

In [0]:
features_df = pd.DataFrame(features)

In [0]:
features_df[0:1440].shape

(1440, 8800)

In [0]:
X_train.shape
y_train[0]

3

In [0]:
label_spe = pd.read_csv('labels.csv')
label_spe = label_spe['emotions']
label_songs = pd.read_csv('labels_songs.csv')
label_songs = label_songs['emotions']

In [0]:
y_final = np.append(label_spe,label_songs)
y_final.shape

(2452,)

In [0]:
features_df.shape

(2452, 8800)

In [0]:
label_songs.shape

(1012,)

In [0]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features_df[1440:], label_songs, test_size=0.1, \
                                                    shuffle = True)


In [0]:
from sklearn.svm import SVC
clf = SVC(gamma=0.001,  kernel='rbf', decision_function_shape='ovo',verbose = True)
clf.fit(X_train, y_train)

[LibSVM]

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=True)

In [0]:
from sklearn.metrics import accuracy_score

In [0]:
pred = clf.predict(X_train)

In [0]:
accuracy_score(y_train, pred)

0.5527472527472528

In [0]:
pred = clf.predict(X_test)
accuracy_score(y_test, pred)

0.45098039215686275

In [0]:
print(__doc__)

import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt


#digits = load_digits()
X, y = X_train,y_train


title = "Learning Curves (Naive Bayes)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=100, test_size=0.1, random_state=0)

estimator = GaussianNB()
plot_learning_curve(estimator, title, X, y, ylim=(0.7, 1.01), cv=cv, n_jobs=4)

title = "Learning Curves (SVM, RBF kernel, $\gamma=0.001$)"
# SVC is more expensive so we do a lower number of CV iterations:
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
estimator = SVC(gamma=0.001)
plot_learning_curve(estimator, title, X, y, (0.7, 1.01), cv=cv, n_jobs=4)

plt.show()